In [1]:
from dask.array import shape
!pip uninstall -y pathos numpy polars h5py

Found existing installation: pathos 0.3.3
Uninstalling pathos-0.3.3:
  Successfully uninstalled pathos-0.3.3
Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: polars 1.17.1
Uninstalling polars-1.17.1:
  Successfully uninstalled polars-1.17.1
Found existing installation: h5py 3.12.1
Uninstalling h5py-3.12.1:
  Successfully uninstalled h5py-3.12.1


In [2]:
!pip install pathos numpy polars h5py scipy

  Using cached pathos-0.3.3-py3-none-any.whl.metadata (11 kB)
  Using cached numpy-2.2.0-cp312-cp312-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached polars-1.17.1-cp39-abi3-macosx_11_0_arm64.whl.metadata (14 kB)
  Using cached h5py-3.12.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (2.5 kB)
Using cached pathos-0.3.3-py3-none-any.whl (82 kB)
Using cached numpy-2.2.0-cp312-cp312-macosx_14_0_arm64.whl (5.1 MB)
Using cached polars-1.17.1-cp39-abi3-macosx_11_0_arm64.whl (28.8 MB)
Using cached h5py-3.12.1-cp312-cp312-macosx_11_0_arm64.whl (2.9 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.0 which is incompatible.
contourpy 1.2.0 requires numpy<2.0,>=1.20, but you have numpy 2.2.0 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.0 which is incompatible.
thi

In [2]:
import polars as pl
import os

In [2]:
# Enable progress bar for lazy execution
pl.Config.set_tbl_cols(100)  # Optional: set table column width
pl.Config.set_tbl_rows(50)  # Optional: set table row height
pl.Config.set_verbose(True)  # Enable verbose output
# pl.Config.set_streaming(True)  # Enable lazy evaluation streaming (if supported)

polars.config.Config

In [3]:
DATA_FOLDER = os.path.join(os.getcwd(), '../data/processed/')

In [4]:
df_ratings_no_text = pl.read_parquet(DATA_FOLDER + 'ratings_no_text.pq')
df_ratings_text = pl.read_parquet(DATA_FOLDER + 'ratings_only_text.pq')

parquet scan with parallel = Columns
parquet scan with parallel = RowGroups


In [3]:
def cleanup_polars(column: pl.Expr):
    return (
        column
        .str.replace_all(r'[^\w ]', '')  # Remove non-alphanumeric characters
        .str.to_lowercase()             # Convert to lowercase
    )

# Convert the DataFrame to a LazyFrame to enable progress tracking
lazy_df = df_ratings_text.lazy()

# Apply the cleanup function
lazy_df = lazy_df.with_columns(
    cleanup_polars(pl.col("text")).alias("text_cleaned")
)

# Collect the results with progress tracking
df_ratings_text = lazy_df.collect(streaming=True, progress_bar=True)


In [4]:
!pip install nltk

In [12]:
!pip install spacy

In [14]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 609.1 kB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [17]:
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 1.0 MB/s eta 0:00:0000:0100:01m
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [18]:
!python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 1.1 MB/s eta 0:00:0000:0100:01m
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


In [19]:
!python -m spacy download de_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 2.8 MB/s eta 0:00:00a 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')


In [5]:
# print all spacy models
!python -m spacy validate

✔ Loaded compatibility table

================= Installed pipeline packages (spaCy v3.8.3) =================
ℹ spaCy installation:
/opt/anaconda3/lib/python3.12/site-packages/spacy

NAME              SPACY            VERSION                            
pl_core_news_sm   >=3.8.0,<3.9.0   3.8.0   ✔
es_core_news_sm   >=3.8.0,<3.9.0   3.8.0   ✔
en_core_web_sm    >=3.8.0,<3.9.0   3.8.0   ✔
fr_core_news_sm   >=3.8.0,<3.9.0   3.8.0   ✔
de_core_news_sm   >=3.8.0,<3.9.0   3.8.0   ✔



In [4]:
import spacy
from tqdm import tqdm
from langdetect import detect
from pathos.multiprocessing import ProcessingPool as Pool
import polars as pl

# Load SpaCy models and disable unnecessary components
spacy_models = {
    'en': spacy.load('en_core_web_sm', disable=['ner', 'parser']),
    'fr': spacy.load('fr_core_news_sm', disable=['ner', 'parser']),
    'es': spacy.load('es_core_news_sm', disable=['ner', 'parser']),
    'de': spacy.load('de_core_news_sm', disable=['ner', 'parser']),
    'pl': spacy.load('pl_core_news_sm', disable=['ner', 'parser']),
}

# Function to process a batch of texts for lemmatization and stopword removal
def batch_lemmatize_remove_stopwords(texts):
    results = []
    for text in texts:
        try:
            lang = detect(text)  # Detect language
            if lang in spacy_models:
                nlp = spacy_models[lang]
                doc = nlp(text)
                results.append(
                    ' '.join(
                        [token.lemma_ for token in doc if not token.is_stop]  # Remove stopwords
                    )
                )
            else:
                results.append(text)  # Return as-is if the language is unsupported
        except Exception:
            results.append(text)  # Return as-is in case of an error
    return results

# Parallel processing function
def parallel_process_lemmatization(df, column, chunk_size, num_workers):
    chunks = [df[column][i:i + chunk_size].to_list() for i in range(0, df.height, chunk_size)]
    tqdm_total = df.height  # Total number of entries for progress tracking

    # Use pathos for multiprocessing
    pool = Pool(num_workers)
    results = []
    for result in tqdm(pool.imap(batch_lemmatize_remove_stopwords, chunks), total=len(chunks), desc="Processing Batches"):
        results.extend(result)

    pool.close()
    pool.join()
    return results


In [5]:
df_ratings_text

text,__index_level_0__,text_cleaned
str,i64,str
"""Puszka 0,33l dzięki Christoph …",0,"""puszka 033l dzięki christoph …"
"""Cerveza pale lager gabonesa. M…",1,"""cerveza pale lager gabonesa mã…"
"""Kolor- złoty, klarowny. Piana …",2,"""kolor złoty klarowny piana dr…"
"""Botella, de GabÃ³n regalo fami…",3,"""botella de gabãn regalo famili…"
"""Many thanks for this beer to E…",4,"""many thanks for this beer to e…"
…,…,…
"""Draft at the source. Clear gol…",7122069,"""draft at the source clear gold…"
"""Tap @brewpub, TiraneClear gold…",7122070,"""tap brewpub tiraneclear golden…"
"""In their beer garden after a w…",7122071,"""in their beer garden after a w…"


In [6]:


# Parameters
chunk_size = 1000  # Adjust chunk size based on memory
num_workers = 4  # Set number of processes

# Process with progress bar
lemmatized_texts = parallel_process_lemmatization(
    df_ratings_text,
    'text_cleaned',
    chunk_size=chunk_size,
    num_workers=num_workers
)

Processing Batches: 100%|██████████| 7103/7103 [1:19:54<00:00,  1.48it/s]


AttributeError: 'DataFrame' object has no attribute 'with_column'

In [7]:

# Add lemmatized results to the Polars DataFrame
df_ratings_text = df_ratings_text.with_columns(
    pl.Series(name='text_cleaned_lemma', values=lemmatized_texts)
)

# Output first few rows to verify
df_ratings_text.head()

text,__index_level_0__,text_cleaned,text_cleaned_lemma
str,i64,str,str
"""Puszka 0,33l dzięki Christoph …",0,"""puszka 033l dzięki christoph …","""puszka 033l dzięki christoph …"
"""Cerveza pale lager gabonesa. M…",1,"""cerveza pale lager gabonesa mã…","""cerveza pale lager gabonés mãs…"
"""Kolor- złoty, klarowny. Piana …",2,"""kolor złoty klarowny piana dr…","""kolor złoty klarowny piać dr…"
"""Botella, de GabÃ³n regalo fami…",3,"""botella de gabãn regalo famili…","""botella gabãn regalo familiar3…"
"""Many thanks for this beer to E…",4,"""many thanks for this beer to e…","""thank beer erzengel pour light…"


In [9]:
df_ratings_text.write_parquet(DATA_FOLDER + 'ratings_text_cleaned_lemma.pq')

In [5]:
df_ratings_text = pl.read_parquet(DATA_FOLDER + 'ratings_text_cleaned_lemma.pq')

parquet scan with parallel = RowGroups


In [28]:
df_ratings_text.filter(pl.col("text_cleaned_lemma").str.contains("head")).limit(10)

dataframe filtered


text,__index_level_0__,text_cleaned,text_cleaned_lemma
str,i64,str,str
"""Many thanks for this beer to E…",4,"""many thanks for this beer to e…","""thank beer erzengel pour light…"
"""Many thanks to Erzengel for sh…",5,"""many thanks to erzengel for sh…","""thank erzengel share pour clea…"
"""330ml bottle. 29-05-2014.From …",8,"""330ml bottle 29052014from gabo…","""330ml bottle 29052014from gabo…"
"""Pours yellow with a large whit…",10,"""pours yellow with a large whit…","""pour yellow large white head …"
"""Can at the DC Local Gathering …",12,"""can at the dc local gathering …","""dc local gathering 812015 shar…"
"""16oz can pours a clear gold wi…",13,"""16oz can pours a clear gold wi…","""16 oz pour clear gold cloudy w…"
"""From a 12 oz bottle at the RB …",14,"""from a 12 oz bottle at the rb …","""12 oz bottle rb aug 1 tasting …"
"""16oz can. Pours a very sad, pa…",16,"""16oz can pours a very sad pale…","""16 oz pour sad pale yellow col…"
"""Can muled from the source by M…",17,"""can muled from the source by m…","""muled source maki thank clear …"


In [6]:
# concat the two dataframes
df_ratings = df_ratings_text.with_columns(
    df_ratings_no_text
)

In [33]:
print(df_ratings.head(5))

shape: (5, 22)
┌─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┐
│ tex ┆ __i ┆ tex ┆ tex ┆ dat ┆ bee ┆ use ┆ bre ┆ abv ┆ sty ┆ rat ┆ pal ┆ tas ┆ app ┆ aro ┆ ove ┆ yea ┆ bre ┆ cou ┆ sta ┆ cou ┆ sta │
│ t   ┆ nde ┆ t_c ┆ t_c ┆ e   ┆ r_i ┆ r_i ┆ wer ┆ --- ┆ le  ┆ ing ┆ ate ┆ te  ┆ ear ┆ ma  ┆ ral ┆ r   ┆ wer ┆ ntr ┆ te_ ┆ ntr ┆ te_ │
│ --- ┆ x_l ┆ lea ┆ lea ┆ --- ┆ d   ┆ d   ┆ y_i ┆ f64 ┆ --- ┆ --- ┆ --- ┆ --- ┆ anc ┆ --- ┆ l   ┆ --- ┆ y_n ┆ y_b ┆ bre ┆ y_u ┆ use │
│ str ┆ eve ┆ ned ┆ ned ┆ dat ┆ --- ┆ --- ┆ d   ┆     ┆ str ┆ f64 ┆ f64 ┆ f64 ┆ e   ┆ f64 ┆ --- ┆ i32 ┆ ame ┆ rew ┆ wer ┆ ser ┆ r   │
│     ┆ l_0 ┆ --- ┆ _le ┆ eti ┆ i64 ┆ i64 ┆ --- ┆     ┆     ┆     ┆     ┆     ┆ --- ┆     ┆ f64 ┆     ┆ --- ┆ ery ┆ y   ┆ --- ┆ --- │
│     ┆ __  ┆ str ┆ mma ┆ me[ ┆     ┆     ┆ i64 ┆     ┆     ┆     ┆     ┆     ┆ f64 ┆     ┆     ┆     ┆ str ┆ --- ┆ --- ┆ str ┆ str │
│     ┆ --- ┆     ┆ --- ┆ ns] ┆     ┆     ┆    

In [13]:
df_ratings.select(pl.col("rating")).min()

rating
f64
0.5


In [7]:
df_lazy = df_ratings.lazy()

In [8]:
# group by rating, then concat all the text_cleaned_lemma
query = (
    df_lazy
    # Select the necessary columns
    .select(["rating", "text_cleaned_lemma"])
    # Tokenize the `text_cleaned_lemma` into words and explode to get one word per row
    .with_columns(
        pl.col("text_cleaned_lemma")
        .str.split(" ")  # Tokenize by splitting on spaces
        .alias("tokens")
    )
    .explode("tokens")  # Expand tokenized list into rows
    # Filter out empty strings (whitespace tokens)
    .filter(pl.col("tokens") != "")
    # Group by rating and tokens (words), count occurrences
    .group_by(["rating", "tokens"])
    .agg(
        pl.len().alias("word_count")
    )
    # Group by rating and collect word counts into a histogram-like structure
    .group_by("rating")
    .agg(
        pl.col("tokens").alias("words"),
        pl.col("word_count").alias("frequencies")
    )
)


In [35]:
# group by country_user, then concat all the text_cleaned_lemma
query = (
    df_lazy
    # Select the necessary columns
    .select(["country_user", "text_cleaned_lemma"])
    # Tokenize the `text_cleaned_lemma` into words and explode to get one word per row
    .with_columns(
        pl.col("text_cleaned_lemma")
        .str.split(" ")  # Tokenize by splitting on spaces
        .alias("tokens")
    )
    .explode("tokens")  # Expand tokenized list into rows
    # Filter out empty strings (whitespace tokens)
    .filter(pl.col("tokens") != "")
    # Group by country and tokens (words), count occurrences
    .group_by(["country_user", "tokens"])
    .agg(
        pl.len().alias("word_count")
    )
    # Group by country and collect word counts into a histogram-like structure
    .group_by("country_user")
    .agg(
        pl.col("tokens").alias("words"),
        pl.col("word_count").alias("frequencies")
    )
)

In [9]:
result = query.collect()

keys/aggregates are not partitionable: running default HASH AGGREGATION
dataframe filtered
estimated unique values: 74841706
estimated unique count: 74841706 exceeded the boundary: 1000, running default HASH AGGREGATION


In [10]:
result

rating,words,frequencies
f64,list[str],list[u32]
3.0,"[""rudys"", ""âgoodyear"", … ""obliteration""]","[1, 1, … 1]"
1.3,"[""podla"", ""snowcone"", … ""dolce""]","[2, 1, … 8]"
0.7,"[""overcarbonated"", ""horkosť"", … ""sprzedaż""]","[4, 1, … 1]"
1.2,"[""alkie"", ""lambicy"", … ""mat""]","[1, 1, … 7]"
2.9,"[""twotenth"", ""ultimi"", … ""biãch""]","[1, 4, … 2]"
2.8,"[""b285099"", ""karamellfargebeige"", … ""norwood""]","[1, 1, … 1]"
2.2,"[""cocoataste"", ""southtaste"", … ""Weissbierkrügerlrated""]","[1, 1, … 1]"
0.5,"[""scifi"", ""politician"", … ""breed""]","[1, 2, … 3]"
5.0,"[""guidã"", ""35337"", … ""grassalfalfa""]","[1, 1, … 1]"


In [15]:
result.write_parquet(DATA_FOLDER + 'word_frequencies_per_rating.pq')

try_get_writeable: local: /Users/alex/Documents/uni/ada/ada-2024-project-sadamasochists/data/processed/word_frequencies_per_rating.pq


In [4]:
result = pl.read_parquet(DATA_FOLDER + 'word_frequencies_per_rating.pq')

parquet scan with parallel = Columns


In [18]:
df_lazy = result.lazy()

In [21]:
banned_words = ["beer"]

query = (
    df_lazy
    .explode(["words", "frequencies"])  # Explode lists into rows
    .filter(~pl.col("words").is_in(banned_words))  # Exclude banned words
    .sort("frequencies", descending=True)  # Sort by frequencies in descending order
    .group_by("rating")
    .agg(
        pl.col("words").first().alias("top_word"),  # Top word by frequency
        pl.col("frequencies").first().alias("frequency")  # Its frequency
    )
)


In [22]:
result_top = query.collect()

dataframe filtered
estimated unique values: 46
run PARTITIONED HASH AGGREGATION


In [29]:
result_top

rating,top_word,frequency
f64,str,u32
0.7,"""taste""",3521
3.7,"""head""",333869
2.3,"""head""",54827
2.6,"""head""",105703
4.6,"""head""",16024
1.3,"""head""",10247
1.6,"""head""",19268
4.2,"""head""",96125
1.8,"""head""",26666


In [44]:
# result_top.write_parquet(DATA_FOLDER + 'top_words_per_country.pq')

In [32]:
words = result_top.sort("rating")

In [33]:
words

rating,top_word,frequency
f64,str,u32
0.5,"""taste""",5812
0.6,"""taste""",2597
0.7,"""taste""",3521
0.8,"""taste""",4280
0.9,"""taste""",5403
1.0,"""taste""",7514
1.1,"""taste""",7133
1.2,"""head""",8670
1.3,"""head""",10247


In [43]:
!pip uninstall numpy -y

Found existing installation: numpy 1.26.0
Uninstalling numpy-1.26.0:
  Successfully uninstalled numpy-1.26.0


In [45]:
!pip install numpy==1.26.4 transformers

  Using cached numpy-1.26.4-cp312-cp312-macosx_11_0_arm64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-macosx_11_0_arm64.whl (13.7 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
blis 1.0.2 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
thinc 8.3.2 requires numpy<2.1.0,>=2.0.0; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.


In [7]:
from transformers import pipeline

In [13]:
!pip install torch

/opt/anaconda3/lib/python3.12/pty.py:95: RuntimeWarning: Using fork() can cause Polars to deadlock in the child process.
In addition, using fork() with Python in general is a recipe for mysterious
deadlocks and crashes.

The most likely reason you are seeing this error is because you are using the
multiprocessing module on Linux, which uses fork() by default. This will be
fixed in Python 3.14. Until then, you want to use the "spawn" context instead.

See https://docs.pola.rs/user-guide/misc/multiprocessing/ for details.

If you really know what your doing, you can silence this warning with the warning module
or by setting POLARS_ALLOW_FORKING_THREAD=1.

  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 MB 5.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 10.1 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.2
    Uninstalling sympy-1.13.2:
      Successfully uninstalled sympy-1.13.2


In [8]:
# Load sentiment analysis pipeline
classifier = pipeline("sentiment-analysis")


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use mps:0


In [14]:
# Function to filter words based on sentiment scores
def filter_sentiment_words(text):
    words = text.split()
    return [word for word in words if classifier(word)[0]['label'] in {"POSITIVE", "NEGATIVE"}]

# Apply to your reviews
df_ratings_with_sentiment = df_ratings.head(100).lazy().with_columns(
    pl.col("text_cleaned_lemma").map_elements(filter_sentiment_words).alias("sentiment_words")
).collect(streaming=True)

RUN STREAMING PIPELINE
[df -> hstack -> ordered_sink]
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` w

In [18]:
# 100 elements takes: 10s
df_ratings.shape

(7102520, 22)

In [19]:
7102520 / 100 * 10 / 60 / 60

197.29222222222222

In [20]:
!pip install -U sentence-transformers

/opt/anaconda3/lib/python3.12/pty.py:95: RuntimeWarning: Using fork() can cause Polars to deadlock in the child process.
In addition, using fork() with Python in general is a recipe for mysterious
deadlocks and crashes.

The most likely reason you are seeing this error is because you are using the
multiprocessing module on Linux, which uses fork() by default. This will be
fixed in Python 3.14. Until then, you want to use the "spawn" context instead.

See https://docs.pola.rs/user-guide/misc/multiprocessing/ for details.

If you really know what your doing, you can silence this warning with the warning module
or by setting POLARS_ALLOW_FORKING_THREAD=1.

  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(tru

In [21]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
embeddings = model.encode(sentences)
print(embeddings)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[[ 0.1884568   0.17425641  0.05447782  0.2905176   0.16766421 -0.04720675
   0.64558     0.15980871  0.22689249 -0.03089059  0.2558835  -0.0525877
  -0.2261015  -0.05710628  0.1304262   0.12495363  0.31749603  0.19444385
  -0.58632535 -0.01258583  0.6099092   0.16432738  0.03331115 -0.27383083
  -0.28975758 -0.2111971  -0.02261382 -0.17035906  0.16159007  0.06082738
  -0.24162409  0.18579188  0.42740935  0.19295171 -0.07234462  0.16611078
   0.10442807  0.20477235  0.21116717  0.19973986 -0.09408248 -0.17383674
   0.06427378  0.28025475 -0.29530594  0.06209547  0.10427693 -0.02364411
   0.12913184 -0.12617461 -0.17898999  0.03700579 -0.61250603  0.05029806
   0.17730357  0.22494122  0.17386065 -0.03840281 -0.21286814  0.25849253
  -0.12101628  0.30971506 -0.4196637   0.0090766   0.14188908 -0.30556944
   0.17621139 -0.07087357 -0.6203312   0.67708343  0.01723743  0.18405125
  -0.1678578   0.20452644 -0.14770268 -0.06175376  0.63017434  0.11120188
   0.05153064  0.1592742  -0.05370892  

In [29]:
sentences = df_ratings.select("text_cleaned_lemma").to_pandas()

In [30]:
sentences = sentences["text_cleaned_lemma"].to_list()

In [31]:
embeddings = model.encode(sentences, show_progress_bar=True)

Batches:   0%|          | 0/221954 [00:00<?, ?it/s]

In [25]:
sentences = df_ratings.head(10000).select("text_cleaned_lemma").to_pandas()

In [32]:
# save embeddings to disk
import numpy as np
np.save(DATA_FOLDER + 'embeddings.npy', embeddings)

In [9]:
import numpy as np
embeddings = np.load(DATA_FOLDER + 'embeddings.npy')

In [1]:
## RELOAD AT THIS POINT BECAUSE THE KERNEL CRASHED TO OFTEN
import polars as pl
import numpy as np
import os

DATA_FOLDER = os.path.join(os.getcwd(), '../data/processed/')

In [7]:
df_no_text = pl.read_parquet(DATA_FOLDER + 'ratings_no_text.pq')
df_ratings_text = pl.read_parquet(DATA_FOLDER + 'ratings_text_cleaned_lemma.pq')

df_ratings = df_ratings_text.with_columns(
    df_no_text
)

In [8]:
df_ratings.head()

text,__index_level_0__,text_cleaned,text_cleaned_lemma,date,beer_id,user_id,brewery_id,abv,style,rating,palate,taste,appearance,aroma,overall,year,brewery_name,country_brewery,state_brewery,country_user,state_user
str,i64,str,str,datetime[ns],i64,i64,i64,f64,str,f64,f64,f64,f64,f64,f64,i32,str,str,str,str,str
"""Puszka 0,33l dzięki Christoph …",0,"""puszka 033l dzięki christoph …","""puszka 033l dzięki christoph …",2016-04-26 12:00:00,410549,175852,3198,5.0,"""Pale Lager""",2.0,2.0,4.0,2.0,4.0,8.0,2016,"""Sobraga""","""Gabon""",null,"""Poland""",null
"""Cerveza pale lager gabonesa. M…",1,"""cerveza pale lager gabonesa mã…","""cerveza pale lager gabonés mãs…",2017-02-17 12:00:00,105273,442761,3198,5.2,"""Pale Lager""",1.9,2.0,4.0,2.0,3.0,8.0,2017,"""Sobraga""","""Gabon""",null,"""Spain""",null
"""Kolor- złoty, klarowny. Piana …",2,"""kolor złoty klarowny piana dr…","""kolor złoty klarowny piać dr…",2016-06-24 12:00:00,105273,288889,3198,5.2,"""Pale Lager""",1.6,2.0,3.0,3.0,3.0,5.0,2016,"""Sobraga""","""Gabon""",null,"""Poland""",null
"""Botella, de GabÃ³n regalo fami…",3,"""botella de gabãn regalo famili…","""botella gabãn regalo familiar3…",2016-01-01 12:00:00,105273,250510,3198,5.2,"""Pale Lager""",1.5,1.0,2.0,4.0,3.0,5.0,2016,"""Sobraga""","""Gabon""",null,"""Spain""",null
"""Many thanks for this beer to E…",4,"""many thanks for this beer to e…","""thank beer erzengel pour light…",2015-10-23 12:00:00,105273,122778,3198,5.2,"""Pale Lager""",1.9,2.0,4.0,2.0,4.0,7.0,2015,"""Sobraga""","""Gabon""",null,"""Germany""",null


In [10]:
df_ratings = df_ratings.with_columns(
    pl.DataFrame({
        "embedding": embeddings
    })
)

In [11]:
print(df_ratings.head())

shape: (5, 23)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ text      ┆ __index_l ┆ text_clea ┆ text_clea ┆ … ┆ state_bre ┆ country_u ┆ state_use ┆ embeddin │
│ ---       ┆ evel_0__  ┆ ned       ┆ ned_lemma ┆   ┆ wery      ┆ ser       ┆ r         ┆ g        │
│ str       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│           ┆ i64       ┆ str       ┆ str       ┆   ┆ str       ┆ str       ┆ str       ┆ array[f3 │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ 2, 384]  │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ Puszka    ┆ 0         ┆ puszka    ┆ puszka    ┆ … ┆ null      ┆ Poland    ┆ null      ┆ [-0.0922 │
│ 0,33l     ┆           ┆ 033l      ┆ 033l      ┆   ┆           ┆           ┆           ┆ 51, 0.12 │
│ dzięki    ┆           ┆ dzięki    ┆ dzięki    ┆   ┆           ┆           

In [28]:
df_ratings.write_parquet(DATA_FOLDER + 'text_with_embeddings.pq')

In [12]:
country_data = df_ratings.filter(pl.col('country_user') == 'Greece')


# Convert embeddings to numpy arrays first
embeddings_list = country_data.select('embedding').to_numpy().flatten().tolist()
country_vectors = np.array([np.array(emb) for emb in embeddings_list])
greece_embeddings = np.mean(country_vectors, axis=0)

In [14]:
greece_embeddings[:9]

array([-0.01107307, -0.0133639 ,  0.14779963,  0.12276171, -0.01576069,
       -0.04867175,  0.410403  , -0.08626071, -0.07765522], dtype=float32)

In [15]:
print("Polars Greece count:", len(country_data))

Polars Greece count: 9344


In [25]:
# Check a single record in Polars
print("First Greece embedding in Polars:")
# Select embedding for text starting with "750 ml bottle at Gueuze"

first_greece = country_data.filter(pl.col('text').str.contains("750 ml bottle at Gueuze tasting 2016, Ulfborg. Thanjks Alex!")).select('embedding').to_numpy()
print(first_greece)

First Greece embedding in Polars:
[[array([-0.13169406, -0.05693194,  0.09207738,  0.05754452,  0.05620266,
          0.04363281,  0.38556316, -0.09819862,  0.16988009, -0.03569547,
         -0.03897982, -0.48118275, -0.32915944,  0.00379987,  0.35807428,
         -0.04372841,  0.12076375,  0.18190432, -0.13706633, -0.04586298,
         -0.17574537,  0.18152708, -0.21668543,  0.10091254, -0.30105105,
          0.08337148, -0.00778008, -0.26100546,  0.02534828,  0.00435741,
          0.23898597,  0.2825697 ,  0.1601582 , -0.08995596,  0.02752494,
         -0.02616315, -0.2775362 , -0.30310145, -0.07825443,  0.23916608,
         -0.02156192, -0.13552654, -0.08342125,  0.09390035, -0.24820158,
         -0.09341329, -0.0695189 , -0.15045294, -0.0010482 ,  0.23539037,
          0.0333218 , -0.28814638, -0.05667413, -0.33202535, -0.02855933,
         -0.08968623, -0.29258746, -0.10395052, -0.1936652 ,  0.21048358,
         -0.10234781, -0.02177402, -0.10794804, -0.06253915, -0.04407612,
    

In [10]:
import numpy as np
import polars as pl
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
from tqdm.auto import tqdm

def analyze_country_text_embeddings(df: pl.DataFrame):
    """
    Analyze textual patterns in beer reviews using embeddings grouped by country.

    Parameters:
    df (pl.DataFrame): Input polars dataframe with beer ratings and embeddings

    Returns:
    dict: Dictionary containing various text-based analyses
    """
    # Calculate average embedding vector per country
    country_embeddings = {}
    unique_countries = df['country_user'].unique().to_list()

    print("Calculating country embeddings...")
    for country in tqdm(unique_countries, desc="Processing countries"):
        country_data = df.filter(pl.col('country_user') == country)
        if country_data.height == 0:
            continue

        # Convert embeddings to numpy arrays first
        embeddings_list = country_data.select('embedding').to_numpy().flatten().tolist()
        country_vectors = np.array([np.array(emb) for emb in embeddings_list])
        country_embeddings[country] = np.mean(country_vectors, axis=0)

    # Remove any countries that had no data
    countries = list(country_embeddings.keys())

    if not countries:
        raise ValueError("No valid countries found in the dataset")

    # Calculate similarity matrix between countries
    similarity_matrix = np.zeros((len(countries), len(countries)))

    print("\nCalculating country similarities...")
    for i, c1 in tqdm(enumerate(countries), total=len(countries), desc="Computing similarities"):
        for j, c2 in enumerate(countries):
            similarity_matrix[i, j] = cosine_similarity(
                country_embeddings[c1].reshape(1, -1),
                country_embeddings[c2].reshape(1, -1)
            )[0, 0]

    # Find most similar countries for each country
    similar_countries = {}
    for i, country in enumerate(countries):
        similarities = [(countries[j], similarity_matrix[i, j])
                        for j in range(len(countries)) if countries[j] != country]
        similar_countries[country] = sorted(similarities, key=lambda x: x[1], reverse=True)[:3]

    # Find representative reviews for each country
    representative_reviews = {}
    print("\nFinding representative reviews...")
    for country in tqdm(countries, desc="Processing reviews"):
        country_data = df.filter(pl.col('country_user') == country)
        if country_data.height == 0:
            continue

        mean_embedding = country_embeddings[country]

        # Calculate similarity to mean embedding for each review
        similarities = []
        for row in country_data.iter_rows(named=True):
            embedding_array = np.array(row['embedding'])
            sim = cosine_similarity(
                mean_embedding.reshape(1, -1),
                embedding_array.reshape(1, -1)
            )[0, 0]
            similarities.append((row['text'], sim))

        # Get top 3 most representative reviews
        representative_reviews[country] = sorted(similarities, key=lambda x: x[1], reverse=True)[:3]

    return {
        'country_embeddings': country_embeddings,
        'similarity_matrix': similarity_matrix,
        'countries': countries,
        'similar_countries': similar_countries,
        'representative_reviews': representative_reviews
    }

def get_country_insights(results, country):
    """
    Get detailed insights for a specific country.
    """
    if country not in results['countries']:
        print(f"Country '{country}' not found in the dataset.")
        print(f"Available countries: {', '.join(sorted(results['countries']))}")
        return

    similar = results['similar_countries'][country]
    reviews = results['representative_reviews'][country]

    print(f"Analysis for {country}:")
    print("\nMost similar countries (by review content):")
    for c, score in similar:
        print(f"- {c}: {score:.3f} similarity")

    print("\nMost representative reviews:")
    for review, score in reviews:
        print(f"\nReview (similarity: {score:.3f}):")
        print(review[:200] + "..." if len(review) > 200 else review)

In [11]:
results = analyze_country_text_embeddings(df_ratings)


Calculating country embeddings...


Processing countries:   0%|          | 0/171 [00:00<?, ?it/s]

/var/folders/3_/4btpstq96m76z23vgv9m8r4m0000gn/T/ipykernel_1506/1343527098.py:23: UserWarning: Comparisons with None always result in null. Consider using `.is_null()` or `.is_not_null()`.
  country_data = df.filter(pl.col('country_user') == country)



Calculating country similarities...


Computing similarities:   0%|          | 0/170 [00:00<?, ?it/s]


Finding representative reviews...


Processing reviews:   0%|          | 0/170 [00:00<?, ?it/s]

In [4]:
# save results
import pickle

with open(DATA_FOLDER + 'country_insights.pkl', 'wb') as f:
    pickle.dump(results, f)

NameError: name 'results' is not defined

In [6]:
# load results
import pickle

with open(DATA_FOLDER + 'country_insights.pkl', 'rb') as f:
    results = pickle.load(f)

EOFError: Ran out of input

In [15]:
get_country_insights(results, "Hungary")

Analysis for Hungary:

Most similar countries (by review content):
- Czech Republic: 0.988 similarity
- Germany: 0.985 similarity
- Spain: 0.985 similarity

Most representative reviews:

Review (similarity: 0.891):
0.5l bottle. Dark orange body with a minimal off-white head. Aroma of toast, a minimal caramel and some fruity, floral hoppyness. The taste is like the aroma, but with more hop bitterness. A really ni...

Review (similarity: 0.888):
0.5l bottle. Dark orange body, white head. Aroma and taste of caramel, toast and a tiny bit of roastyness. The taste also has some dark fruits and some floral hoppyness. OK beer.

Review (similarity: 0.886):
Bottle. Clear deep yellow/orange color with a small white head, disappearing. Aroma is malty, sweet. Taste is malty, grassy hops, with a nice bitterness, maybe too much for a so called lager. Medium c...
